In [1]:
import pandas as pd
#Importar pyomo
from Funciones_Hub_Nutresa_MultiHub import *
import datetime
import os
from pulp import *
import random


In [ ]:
# Función que crea un diccionario para almacenar cada hoja del archivo de Excel en un dataframe
def Diccionario_dataframes_desde_excel(ruta_carpeta,nombre_archivo):
    
    """
    Función que crea un diccionario para almacenar cada hoja del archivo de Excel en un dataframe.
    
    Argumentos:
        ruta_carpeta (string): Ruta donde está ubicado el archivo de excel con los datos de entrada para la instancia o escenario que se desea modelar.
                             Ejemplo: 'C:\\Users\\Usuario1\\FolderModelo'
                             
        nombre_archivo (string):  Nombre del archivo de excel con los datos de entrada para la instancia o escenario que se desea modelar.                 
                            Ejemplo: 'Instancia1 PL.xlsx'
    
    Returns:
        dict: Diccionario con un dataframe por cada hoja del archivo de Excel.
    """           
    
    ruta_instancia=ruta_carpeta +r'\\' + nombre_archivo
    try:
        # Leer el archivo completo de Excel con todo su contenido
        excel_data = pd.read_excel(ruta_instancia, sheet_name=None)   
        
        # Crear un diccionario para almacenar cada hoja del archivo de Excel en un dataframe
        dfs = {}
        
        # Iterar sobre cada pestaña (hoja) del archivo de Excel
        for sheet_name, data in excel_data.items():
            # Crear un dataframe por cada hoja del archivo
            dfs[sheet_name] = pd.DataFrame(data)
        
        # Imprimir el nombre de cada hoja y los primeros 3 registros de cada dataframe creado (opcional)
        #for sheet_name, df in dfs.items():
        #    print(f"DataFrame for sheet: {sheet_name}")
        #    print(df.head(3))  # Print the first few rows of each DataFrame
            
        # Obtener la lista de dataframes creados
        #list_of_dfs = list(dfs.values())

        # Obtener la lista de los nombres de cada hoja del archivo de excel
        #list_of_sheet_names = list(dfs.keys())
        
        return dfs
    except Exception as e:
        print(f"Error al leer el archivo de Excel: {e}")
        return None


In [ ]:
# Especificar la ruta de los archivos de excel que contienen los inputs
Ruta_Carpeta_Instancia = r"\DFQ"
#Archivo_instancia_opt='Instancia5_Todos los Escenarios de Optimización Proyección.xlsx'
Archivo_inputs='Datos pruebas.xlsx'
inputs_opt_dfq=Diccionario_dataframes_desde_excel(Ruta_Carpeta_Instancia,Archivo_inputs)


Definición de conjuntos

In [3]:
Plantas=list(set(inputs_opt_dfq['Escalones MOD']['Planta']))
Recursos=list(set(inputs_opt_dfq['Escalones MOD']['Recurso']))
Materiales=list(set(inputs_opt_dfq['Demanda']['Material']))
Materia_prima_carnica=list(set(inputs_opt_dfq['MPC x PT']['Materia Prima Carnica']))
Materia_prima_carnica_importada=list(set(inputs_opt_dfq['MPCI x PT']['Materia Prima Carnica Importada']))
Materia_prima_no_carnica=list(set(inputs_opt_dfq['MPNC x PT']['Materia Prima NO Carnica']))
Cedis=list(set(inputs_opt_dfq['Demanda']['Cedi']))
Origenes = list(set(Cedis+Plantas))
BOR = list(set(inputs_opt_dfq['BOR']['BOR']))
Escalones_MOD=list(set(inputs_opt_dfq['Escalones MOD']['Escalon']))

Definición de parametros

In [4]:
def crear_diccionario(df, columnas_clave, columna_valor):
    """
    Crea un diccionario a partir de un DataFrame, usando las columnas especificadas como clave y valor.
    
    :param df: DataFrame de pandas que contiene los datos.
    :param columnas_clave: Lista de nombres de las columnas que formarán la clave del diccionario.
    :param columna_valor: Nombre de la columna que formará el valor del diccionario.
    
    :return: Un diccionario con claves formadas por las columnas especificadas y valores correspondientes.
    """
    diccionario = {}
    for index, row in df.iterrows():
        if len(columnas_clave) == 1:
            clave = row[columnas_clave[0]]
        else:
            clave = tuple(row[col] for col in columnas_clave)
        valor = row[columna_valor]
        diccionario[clave] = valor
    return diccionario

In [5]:

Demanda = crear_diccionario(inputs_opt_dfq['Demanda'],['Cedi','Material'],'Cantidad')
MPC_x_PT = crear_diccionario(inputs_opt_dfq['MPC x PT'],['Materia Prima Carnica','Planta','BOR','Material'],'Cantidad')
MPCI_x_PT = crear_diccionario(inputs_opt_dfq['MPCI x PT'],['Materia Prima Carnica Importada','Planta','BOR','Material'],'Cantidad')
MPNC_x_PT = crear_diccionario(inputs_opt_dfq['MPNC x PT'],['Materia Prima NO Carnica','Planta','BOR','Material'],'Cantidad')
Costo_Variable = crear_diccionario(inputs_opt_dfq['Costo Variable'],['Planta','Recurso','Material'],'Valor')
Unidades_x_Canasta = crear_diccionario(inputs_opt_dfq['Unidades x Canasta'],['Material'],'Valor')
Unidades_x_Canasta={clave: 1/valor for clave, valor in Unidades_x_Canasta.items()} #Se genera esta división para que en las restricciones se deba multiplicar y no dividir
Costo_x_Unidad_MPNC = crear_diccionario(inputs_opt_dfq['Costo x Unidad MPNC'],['Materia Prima NO Carnica','Planta'],'Valor')
Costo_x_Unidad_MPC = crear_diccionario(inputs_opt_dfq['Costo x Unidad MPC'],['Materia Prima Carnica', 'Planta'],'Valor')
Costo_x_Unidad_MPCI = crear_diccionario(inputs_opt_dfq['Costo x Unidad MPCI'],['Materia Prima Carnica Importada', 'Planta'],'Valor')
MPNC_Puesta_en_planta = crear_diccionario(inputs_opt_dfq['Costo x Unidad MPNC'],['Materia Prima NO Carnica','Planta'],'Puesto en planta')
MPC_Puesta_en_planta = crear_diccionario(inputs_opt_dfq['Costo x Unidad MPC'],['Materia Prima Carnica', 'Planta'],'Puesto en planta')
MPCI_Puesta_en_planta = crear_diccionario(inputs_opt_dfq['Costo x Unidad MPCI'],['Materia Prima Carnica Importada', 'Planta'],'Puesto en planta')

Costo_flete_mpci_a_Planta = crear_diccionario(inputs_opt_dfq['Costo flete MPCI a Planta'],['Planta'],'Valor')
Capacidad_Vehiculo_MPCI = crear_diccionario(inputs_opt_dfq['Costo flete MPCI a Planta'],['Planta'],'Capacidad')
Costo_flete_mpc_a_Planta = crear_diccionario(inputs_opt_dfq['Costo flete MPC a Planta'],['Planta'],'Valor')
Capacidad_Vehiculo_MPC = crear_diccionario(inputs_opt_dfq['Costo flete MPC a Planta'],['Planta'],'Capacidad')
Costo_flete_mpnc_a_Planta = crear_diccionario(inputs_opt_dfq['Costo flete MPNC a Planta'],['Planta'],'Valor')
Capacidad_Vehiculo_MPNC = crear_diccionario(inputs_opt_dfq['Costo flete MPNC a Planta'],['Planta'],'Capacidad')

Costo_ruta = crear_diccionario(inputs_opt_dfq['Arcos Transporte Primario'],['Origen', 'Cedi'],'Costo habilitar frecuencia adicional')
Capacidad_ruta = crear_diccionario(inputs_opt_dfq['Arcos Transporte Primario'],['Origen', 'Cedi'],'Capacidad de frecuencia habilitable')
Ruta_activa = crear_diccionario(inputs_opt_dfq['Arcos Transporte Primario'],['Origen', 'Cedi'],'Habilitado')

Tasa_de_produccion = crear_diccionario(inputs_opt_dfq['Tasa de produccion'],['Planta','Recurso','BOR'],'Cantidad')
BOR_material = crear_diccionario(inputs_opt_dfq['BOR'],['Planta','BOR','Material'],'BOR Activa')

Costo_escalon = crear_diccionario(inputs_opt_dfq['Escalones MOD'],['Planta','Recurso','Escalon'],'Costo escalon')
Horas_maximos= crear_diccionario(inputs_opt_dfq['Escalones MOD'],['Planta','Recurso','Escalon'],'Horas maximas')

**Creación del modelo**

In [6]:
modelo = LpProblem("CostoProyectado",LpMinimize)

Variables de desición

In [7]:

##Cantidad a producir del material m, en planta p en el recurso r
Horas_activos = LpVariable.dicts('Horas_activos',[(p,r) for p in Plantas for r in Recursos],lowBound=0,cat='Integer')
##Cantidad a producir del material m, en planta p en el recurso r
Vehiculos_mpc = LpVariable.dicts('Vehiculos_mpc',[(p) for p in Plantas],lowBound=0,cat='Integer')
##Cantidad a producir del material m, en planta p en el recurso r
Vehiculos_mpci = LpVariable.dicts('Vehiculos_mpci',[(p) for p in Plantas],lowBound=0,cat='Integer')
##Cantidad a producir del material m, en planta p en el recurso r
Vehiculos_mpnc = LpVariable.dicts('Vehiculos_mpnc',[(p) for p in Plantas],lowBound=0,cat='Integer')
##Cantidad a usar de materia prima carnica mp en la planta p
Meteria_prima_carnica_planta = LpVariable.dicts('Meteria_prima_carnica_planta',[(mp,p) for mp in Materia_prima_carnica for p in Plantas],lowBound=0,cat='Continuous')
##Cantidad a usar de materia prima carnica importada mpi en la planta p
Meteria_prima_carnica_importada_planta = LpVariable.dicts('Meteria_prima_carnica_importada_planta',[(mpi,p) for mpi in Materia_prima_carnica_importada for p in Plantas],lowBound=0,cat='Continuous')
##Cantidad a usar de materia prima no carnica mn en la planta p
Meteria_prima_no_carnica_planta = LpVariable.dicts('Meteria_prima_no_carnica_planta',[(mn,p) for mn in Materia_prima_no_carnica for p in Plantas],lowBound=0,cat='Continuous')
##Cantidad a enviar del material m, desde origen o al cedi c
Cantidad_enviar = LpVariable.dicts('Cantidad_enviar',[(m,o,c) for m in Materiales for o in Origenes for c in Cedis],lowBound=0,cat='Continuous')
##Cantidad a recibir del material m,en el cedi c
Cantidad_recibir = LpVariable.dicts('Cantidad_recibir',[(m,c) for m in Materiales for c in Cedis],lowBound=0,cat='Continuous')
##Vehiculos a enviar desde origen o al cedi c
Vehiculos_Cedi = LpVariable.dicts('Vehiculos_Cedi',[(o,c) for o in Origenes for c in Cedis],lowBound=0,cat='Integer')

Cantidad_producir = LpVariable.dicts('Cantidad_producir',[(p,b,m) for m in Materiales for b in BOR for p in Plantas],lowBound=0,cat='Continuous')
Cantidad_pasar_recurso = LpVariable.dicts('Cantidad_pasar_recurso',[(p,b,r,m) for r in Recursos for b in BOR for p in Plantas for m in Materiales],lowBound=0,cat='Continuous')
producir_BOR = LpVariable.dicts('producir_BOR',[(p,b) for b in BOR for p in Plantas],lowBound=0,cat='Continuous')

## Binaria que me indica que escalon voy a prender
Escalones_activos = LpVariable.dicts('Escalones_activos',[(p,r,e) for p in Plantas for r in Recursos for e in Escalones_MOD],lowBound=0,cat='Binary')

Funcion Objetivo

In [8]:
Costo_MOD = LpVariable("Costo Mano de obra")
Costo_variable = LpVariable("Costo variable")
Transporte_Cedi = LpVariable("Costo de transporte pt a CEDI")
Transporte_mpci_planta = LpVariable("Costo de transporte mpci a planta")
Transporte_mpc_planta = LpVariable("Costo de transporte mpc a planta")
Transporte_mpnc_planta = LpVariable("Costo de transporte mpnc a planta")
Costo_MPCI_planta = LpVariable("Costo de usar MPCI en planta")
Costo_MPC_planta = LpVariable("Costo de usar MPC en planta")
Costo_MPNC_planta = LpVariable("Costo de usar MPNC en planta")
costototal = LpVariable("Costo total")

In [9]:
# modelo+= Costo_MOD == lpSum(Turnos_activos[p,r]*Costo_turnos[p,r]  for p in Plantas for r in Recursos ) 
modelo+= Costo_MOD == lpSum(Escalones_activos[p,r,e]*Costo_escalon.get((p,r,e),0)  for p in Plantas for r in Recursos for e in Escalones_MOD) 
modelo+= Costo_variable == lpSum(Cantidad_pasar_recurso[p,b,r,m]*Costo_Variable.get((p,r,m),0) for b in BOR for m in Materiales for r in Recursos for p in Plantas ) 
modelo+= Transporte_mpc_planta == lpSum(Vehiculos_mpc[p]*Costo_flete_mpc_a_Planta[p] for p in Plantas )
modelo+= Transporte_mpci_planta == lpSum(Vehiculos_mpci[p]*Costo_flete_mpci_a_Planta[p] for p in Plantas )
modelo+= Transporte_mpnc_planta == lpSum(Vehiculos_mpnc[p]*Costo_flete_mpnc_a_Planta[p] for p in Plantas )
modelo+= Costo_MPC_planta == lpSum(Meteria_prima_carnica_planta[mp,p]*Costo_x_Unidad_MPC[mp,p] for mp in Materia_prima_carnica for p in Plantas)
modelo+= Costo_MPCI_planta == lpSum(Meteria_prima_carnica_importada_planta[mpi,p]*Costo_x_Unidad_MPCI[mpi,p] for mpi in Materia_prima_carnica_importada for p in Plantas)
modelo+= Costo_MPNC_planta == lpSum(Meteria_prima_no_carnica_planta[mn,p]*Costo_x_Unidad_MPNC[mn,p] for mn in Materia_prima_no_carnica for p in Plantas )
modelo+= Transporte_Cedi == lpSum(Vehiculos_Cedi[o,c]*Costo_ruta.get((o,c),0) for c in Cedis for o in Origenes )

modelo+= costototal == Costo_MOD + Costo_variable + Transporte_Cedi + Transporte_mpc_planta + Costo_MPC_planta + Costo_MPNC_planta + Transporte_mpnc_planta +Transporte_mpci_planta + Costo_MPCI_planta
modelo+= costototal

**Restricciones**

Restricciones nuevas o con cambios

In [10]:
# # Sumar turnos activos de todos los recursos
for p in Plantas:
    for r in Recursos:
        # if Tasa_de_produccion.get((p,r,b),0):            
            modelo += lpSum(Cantidad_pasar_recurso[p,b,r,m] * (1 /Tasa_de_produccion.get((p,r,b),0.00001)) for b in BOR for m in Materiales) <= Horas_activos[p,r] 

# # Cantidad a producir usando la BOR b en la planta p, ligado al recurso con menor capacidad
for p in Plantas:    
    for b in BOR:
        for r in Recursos:
            if Tasa_de_produccion.get((p,r,b),0):
                modelo += producir_BOR[p,b]  <= Tasa_de_produccion.get((p,r,b),0) * Horas_activos[p,r] , f"Cantidad producir usando la BOR {b} en la planta {p}, ligado al recurso {r} con menor capacidad identificador {random.randint(1, 10000)}" 

# # Cantidad de a producir usando la BOR b en la planta p en el recurso r 
for p in Plantas:
    for b in BOR:
        for r in Recursos:
            for m in Materiales:
                if Tasa_de_produccion.get((p,r,b),0):
                    modelo += producir_BOR[p,b] * (BOR_material.get((p,b,m),0)) == Cantidad_pasar_recurso[p,b,r,m] ,f"Cantidad de a producir usando la BOR {b} en la planta {p} en el recurso {r}  identificador {random.randint(1, 10000)}" 

# # cantidad producida del material m es mayor a la cantidad enviada  a todos los cedis desde la planta p
for m in Materiales:
    for p in Plantas:
        modelo += lpSum(Cantidad_producir[p,b,m] for b in BOR) >= lpSum(Cantidad_enviar[m,p,c] for c in Cedis) , f"cantidad producida del material {m} es mayor a la demanda identificador {random.randint(1, 10000)}" 

# # Cantidad a producir del material m usando la BOR b en la planta p
for p in Plantas:
    for b in BOR:
        for m in Materiales:
             modelo += producir_BOR[p,b] * (BOR_material.get((p,b,m),0))  >= Cantidad_producir[p,b,m] ,f"# # Cantidad a producir del material {m} usando la BOR {b} en la planta {p} identificador {random.randint(1, 10000)}" 

# # Activar horas dentro de los escalones habilitados
for p in Plantas:
    for r in Recursos:         
        modelo += Horas_activos[p,r] <= lpSum(Horas_maximos.get((p,r,e),0) * Escalones_activos[p,r,e] for e in Escalones_MOD), f"# # escalones activos planta  {p} recurso {r} identificador {random.randint(1, 10000)}" 
  
#Solo puedo activar un escalon
for p in Plantas:
    for r in Recursos:         
       modelo += 1 >= lpSum(Escalones_activos[p,r,e] for e in Escalones_MOD), f"## máximo escalones activosplanta  {p} recurso {r} identificador {random.randint(1, 10000)}" 

# Consumo de mpnc 
for p in Plantas:
    for mn in Materia_prima_no_carnica:
        modelo +=  lpSum(Cantidad_producir[p,b,m] * MPNC_x_PT.get((mn,p,b,m),0) for m in Materiales for b in BOR) <= Meteria_prima_no_carnica_planta[mn,p] , f"Materia prima no carnica {mn} en planta {p} identificador {random.randint(1, 100)}" 
                            
# Cantidad de vehiculos MPNC a enviar, se resta 1 porque si es puesta en planta no deberia ocupar capacidad de los vehiculos 
for p in Plantas:
    modelo +=  lpSum(Meteria_prima_no_carnica_planta[mn,p] * (1-MPNC_Puesta_en_planta[mn,p]) for mn in Materia_prima_no_carnica) <= Vehiculos_mpnc[p] * Capacidad_Vehiculo_MPNC[p] , f" vehiculos mpnc a planta {p} identificador {random.randint(1, 100)}" 
   
# Consumo de mpc 
for p in Plantas:
    for mp in Materia_prima_carnica:
        modelo +=  lpSum(Cantidad_producir[p,b,m] * MPC_x_PT.get((mp,p,b,m),0) for m in Materiales for b in BOR) <= Meteria_prima_carnica_planta[mp,p] , f"Materia prima  carnica {mp} en planta {p} identificador {random.randint(1, 100)}" 
            
# Cantidad de vehiculos MPC a enviar, se resta 1 porque si es puesta en planta no deberia ocupar capacidad de los vehiculos  
for p in Plantas:
    modelo +=  lpSum(Meteria_prima_carnica_planta[mp,p] * (1-MPC_Puesta_en_planta[mp,p]) for mp in Materia_prima_carnica) <= Vehiculos_mpc[p] * Capacidad_Vehiculo_MPC[p] , f" vehiculos mpc a planta {p} identificador {random.randint(1, 100)}" 
                
# Consumo de mpci 
for p in Plantas:
    for mpi in Materia_prima_carnica_importada:
        modelo +=  lpSum(Cantidad_producir[p,b,m] * MPCI_x_PT.get((mpi,p,b,m),0) for m in Materiales for b in BOR) <= Meteria_prima_carnica_importada_planta[mpi,p] , f"Materia prima  carnica {mpi} en planta {p} identificador {random.randint(1, 100)}" 
            
# Cantidad de vehiculos MPCi a enviar, se resta 1 porque si es puesta en planta no deberia ocupar capacidad de los vehiculos  
for p in Plantas:
    modelo +=  lpSum(Meteria_prima_carnica_importada_planta[mpi,p] * (1-MPCI_Puesta_en_planta[mpi,p]) for mpi in Materia_prima_carnica_importada) <= Vehiculos_mpci[p] * Capacidad_Vehiculo_MPCI[p] , f" vehiculos mpci a planta {p} identificador {random.randint(1, 100)}" 
            
#recibo en cedi todo lo que se envia desde todos los origenes
for c in Cedis:
    for m in Materiales:
        modelo += Cantidad_recibir[m,c] == lpSum(Cantidad_enviar[m,o,c] * Ruta_activa.get((o,c),0) for o in Origenes) , f" Cantidad de material {m} recibido en Cedi {c} {random.randint(1, 100)}"  


#envio menos de lo que recibo y menos la demanda del cedi
for o in Cedis:
    for m in Materiales:
        modelo += lpSum(Cantidad_enviar[m,o,c] for c in Cedis) <= Cantidad_recibir[m,o] - Demanda.get((o,m),0) , f"cantidad a enviar del material {m} desde el origen {o} identificador {random.randint(1, 100)}" 

#Recibo minimo la demanda
for c in Cedis:
    for m in Materiales:
       modelo +=  Demanda.get((c,m),0) <= Cantidad_recibir[m,c]  , f"Cantidad a recibir del material {m} en el cedi {c}"

# Calculo cantidad de vehiculos
for o in Origenes:
    for c in Cedis:
        modelo += Capacidad_ruta.get((o,c),0)*Vehiculos_Cedi[o,c] >= lpSum(Cantidad_enviar[m,o,c]*Unidades_x_Canasta[m] for m in Materiales) , f"Cantidad de vehiculos de {o} a {c}"
 

In [11]:
# Supongamos que ya has resuelto el modelo
modelo.solve()

# Obtener el estado del modelo
estado_modelo = LpStatus[modelo.status]
print(estado_modelo)
costo_operacion = value(modelo.objective)

# Imprimir el estado del modelo y el costo de la operación
print("Status:", estado_modelo)
print("Costo de la operación = ", costo_operacion)

# Extraer valores de las variables de decisión
valores_variables = [{'Variable': v.name, 'Valor': v.varValue} for v in modelo.variables()]

# Extraer precios sombra y holguras de las restricciones
sensibilidad = [{'Restricción': i, 'Precio sombra': j.pi, 'Holgura': j.slack}
                for i, j in modelo.constraints.items()]

# Convertir a DataFrames
df_variables = pd.DataFrame(valores_variables)
df_sensibilidad = pd.DataFrame(sensibilidad)

# Guardar los resultados en un archivo Excel
with pd.ExcelWriter('resultados_optimización.xlsx') as writer:
    # Guardar estado del modelo y costo de operación
    pd.DataFrame({'Estado del modelo': [estado_modelo], 'Costo de la operación': [costo_operacion]}).to_excel(writer, sheet_name='Resumen', index=False)
    # Guardar valores de las variables
    df_variables.to_excel(writer, sheet_name='Valores Variables', index=False)
    # Guardar precios sombra y holguras
    df_sensibilidad.to_excel(writer, sheet_name='Sensibilidad', index=False)

print("Resultados guardados en 'resultados_optimización.xlsx'")

Optimal
Status: Optimal
Costo de la operación =  81.0
Resultados guardados en 'resultados_optimización.xlsx'
